In [210]:
%reset
import httpx
import re
import requests
import codecs
import time
import nltk

import threading
import pandas as pd

from bs4 import BeautifulSoup as bs

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options

import yfinance as yf


Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [211]:
# #constants
# HEADERS = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
#            'Accept-Encoding': 'gzip, deflate, br',
#            'Accept-Language': 'en-US,en;q=0.9,nl;q=0.8',
#            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'}

# DATA_LOCATION = 'Datasets/'

# HOMEPAGE_URL = 'https://www.investopedia.com/markets-news-4427704'

# VUSA_DOWNLOAD_URL = 'https://datahub.io/core/s-and-p-500-companies'
# SP_DOWNLOAD_URL = 'https://datahub.io/core/s-and-p-500'
# DATAHUB_URL = 'https://datahub.io'

# YAHOO_URL = 'https://finance.yahoo.com/'

# #don
# nltk.download(['punkt', 'wordnet', 'stopwords', 'omw-1.4', 'vader_lexicon'])

# options = Options()
# options.add_argument('--headless')
# options.add_argument('--disable-gpu') 

# soup = get_response(HOMEPAGE_URL, HEADERS)

# titles = get_card_titles()
# urls = get_card_urls()

# investopedia_df = pd.DataFrame({'url': urls, 'title': titles})

# articles, raw_articles = get_url_content(investopedia_df['url'])

# investopedia_df['article'] = articles

# symbols = get_symbols(raw_articles, 'widgetsymbol')

# investopedia_df['symbols'] = symbols

# #holdings
# get_datahub_csv(VUSA_DOWNLOAD_URL, 'vusa_holdings.csv')

# #publieke dataset
# get_datahub_csv(SP_DOWNLOAD_URL, 'time_data.csv')

In [212]:
# options = Options()
# options.add_argument('--headless')
# options.add_argument('--disable-gpu')

In [213]:
# def get_response(url, cookies={}, headers=HEADERS):
#     r = httpx.get(url, cookies=cookies, headers=headers, follow_redirects=True)
#     soup = bs(r.content)
#     return soup

In [214]:
# soup = get_response(HOMEPAGE_URL, HEADERS)

In [215]:
# def get_card_titles():
#     titles = []
#     a_tags = soup.find_all('a', class_='card')
#     for i in range(len(a_tags)):
#         titles.append(a_tags[i].span.text)
#     return titles

# def get_card_urls():
#     urls = []
#     a_tags = soup.find_all('a', class_='card')
#     for i in range(len(a_tags)):
#         urls.append(a_tags[i]['href'])
#     return urls

In [216]:
# titles = get_card_titles()
# urls = get_card_urls()

In [217]:
# investopedia_df = pd.DataFrame({'url': urls, 'title': titles})
# display(investopedia_df)

In [218]:
# def clean_text(text):
#     return re.sub(r'[^\w\s]', '', text)

In [219]:
# def get_url_content(urls):
#     articles = []
#     raw_articles = []
#     for url in urls:
#         soup = get_response(url)
#         paragraphs = soup.find('div', class_='article-content').find_all('p')
#         article = ''.join([p.get_text(separator=' ') for p in paragraphs])
#         raw_articles.append(paragraphs)
#         articles.append(clean_text(article))
        
#     return articles, raw_articles

# def get_symbols(raw_articles, href_ref):
#     symbol_list = []
#     for p_list in raw_articles:
#         article_symbols = []
#         for p in p_list:
#             symbols = [a_tag.text for a_tag in p.find_all('a') if re.search(href_ref, a_tag['href'])]
#             if symbols:
#                 for symbol in symbols:
#                     article_symbols.append(symbol)
#         symbol_list.append(article_symbols)
#     return symbol_list

In [220]:
# articles, raw_articles = get_url_content(investopedia_df['url'])

In [221]:
# investopedia_df['article'] = articles

In [222]:
# symbols = get_symbols(raw_articles, 'widgetsymbol')

In [223]:
# investopedia_df['symbols'] = symbols

In [224]:
# display(investopedia_df)

In [225]:
# def get_datahub_csv(URL, filename):
#     soup = get_response(URL)
#     download_url = soup.find_all('table')[1].find_all('a')[1]['href']
#     download_url = DATAHUB_URL + download_url
#     with open(DATA_LOCATION + filename, 'wb') as f:
#         f.write(requests.get(download_url).content)

In [226]:
# get_datahub_csv(VUSA_DOWNLOAD_URL, 'vusa_holdings.csv')
# get_datahub_csv(SP_DOWNLOAD_URL, 'time_data.csv')

In [227]:
# vusa_df = pd.read_csv(DATA_LOCATION + 'vusa_holdings.csv')
# symbols_in_vusa = vusa_df['Symbol'].tolist()

In [228]:
#investopedia filteren op vusa symbols
# investopedia_df = investopedia_df[investopedia_df['symbols'].apply(lambda symbols: any(symbol in symbols_in_vusa for symbol in symbols))]
# #combine dataframes
# df = combine_dfs(investopedia_df, yahoo_news_df)

# #clean and tokenize the articles
# df['tokenized_article'] = clean_and_tokenize(df['article'])

# #do sentiment analysis
# analyzer = SentimentIntensityAnalyzer()
# df['positivity_score'] = [analyzer.polarity_scores(article)['pos'] for article in df['article']]
# df['negativity_score'] = [analyzer.polarity_scores(article)['neg'] for article in df['article']]

In [229]:
# def get_article_links(driver):
#     links = driver.find_elements(By.TAG_NAME, 'a')
#     return links

In [230]:
# def get_yahoo_news_articles_df():
#     yahoo_news_driver = webdriver.Chrome(options=options)

#     yahoo_news_driver.get(YAHOO_URL)
#     time.sleep(1)
#     yahoo_news_driver.find_element(By.CLASS_NAME, 'reject-all').click()
#     actions = ActionChains(yahoo_news_driver)
#     for i in range(1500):
#         actions.scroll_by_amount(0, 500)
#     actions.perform()
#     actions.reset_actions()

#     links = get_article_links(yahoo_news_driver)
#     df = pd.DataFrame({'title': [link.text for link in links], 
#                        'url': [link.get_attribute('href') for link in links], 
#                        'classes': [link.get_attribute('class') for link in links]})
    
#     yahoo_news_driver.quit()
#     df = df[df['url'] \
#             .str.contains('/news/') & df['classes'].str.contains('js-content-viewer')]\
#             .drop_duplicates(subset='url', keep='last')\
#             .reset_index(drop=True)
#     return df

In [231]:
# yahoo_news_df = get_yahoo_news_articles_df()

In [232]:
# display(yahoo_news_df)

In [233]:
# def progress_bar(current, total, bar_length=20):
#     fraction = current / total

#     arrow = int(fraction * bar_length - 1) * '-' + '>'
#     padding = int(bar_length - len(arrow)) * ' '

#     ending = '\n' if current == total else '\r'

#     print(f'Progress: [{arrow}{padding}] {int(fraction*100)}%', end=ending)

In [234]:
# def scrape_articles(urls):
#     articles = []
#     raw_articles = []
#     yahoo_driver = webdriver.Chrome(options=options)
#     yahoo_driver.get(YAHOO_URL)
#     yahoo_driver.find_element(By.CLASS_NAME, 'reject-all').click()

#     for index, url in enumerate(urls):
#         yahoo_driver.get(url)
#         try:
#             yahoo_driver.find_element(By.CLASS_NAME, 'collapse-button').click()
#         except:
#             pass
#         article_p_elements = yahoo_driver.find_element(By.CLASS_NAME, 'caas-body').find_elements(By.TAG_NAME, 'p')
#         article_by_paragraph = [p.text for p in article_p_elements if p.text!='']
        
#         raw_article = [bs(element.get_attribute('outerHTML'), "html.parser") for element in article_p_elements]
#         article = " ".join(article_by_paragraph)
#         clean_article = clean_text(article)
#         articles.append(clean_article)
#         raw_articles.append(raw_article)
#         progress_bar(index+1, len(urls))
    
        
#     yahoo_driver.quit()
    
#     return articles, raw_articles

In [235]:
# yahoo_news_articles, raw_yahoo_news_articles = scrape_articles(yahoo_news_df['url'])

In [236]:
# yahoo_news_df['article'] = yahoo_news_articles

In [237]:
# yahoo_news_df

In [238]:
# def combine_dfs(investopedia_df, yahoo_df):
#     df1 = investopedia_df.drop(['symbols'], axis=1)
#     df2 = yahoo_df.drop('classes', axis=1)
#     article_df = pd.concat([df1, df2])
#     article_df = article_df.reset_index(drop=True)
#     return article_df

In [239]:
# display(investopedia_df)
# df = combine_dfs(investopedia_df, yahoo_news_df)

In [240]:
# def clean_and_tokenize(articles):
#     lemmatizer = WordNetLemmatizer()
    
#     tokenized_articles = []
#     for article in articles:
#         tokenized_article = nltk.word_tokenize(article)
#         tokenized_article = [token for token in tokenized_article if token not in stopwords.words('english')]
#         tokenized_article = [lemmatizer.lemmatize(token) for token in tokenized_article]
        
#         tokenized_articles.append(tokenized_article)
        
#     return tokenized_articles

In [241]:
# df['tokenized_article'] = clean_and_tokenize(df['article'])

In [242]:
# df

In [243]:
# analyzer = SentimentIntensityAnalyzer()
# df['positivity_score'] = [analyzer.polarity_scores(article)['pos'] for article in df['article']]
# df['negativity_score'] = [analyzer.polarity_scores(article)['neg'] for article in df['article']]

In [244]:
# pos = df.describe().T[:1].reset_index(drop=True)
# neg = df.describe().T[1:].reset_index(drop=True)

# pos.columns = ['pos_' + col for col in pos.columns]
# neg.columns = ['neg_' + col for col in neg.columns]

# pd.concat([pos, neg], axis=1)

In [245]:
# new_df2 = pd.read_csv(DATA_LOCATION + 'time_data.csv')

In [246]:
# new_df2

In [272]:
class Pipeline:
    def __init__(self):
        self.HEADERS = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
           'Accept-Encoding': 'gzip, deflate, br',
           'Accept-Language': 'en-US,en;q=0.9,nl;q=0.8',
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'}

        self.DATA_LOCATION = 'Datasets/'

        self.HOMEPAGE_URL = 'https://www.investopedia.com/markets-news-4427704'

        self.VUSA_DOWNLOAD_URL = 'https://datahub.io/core/s-and-p-500-companies'
        self.SP_DOWNLOAD_URL = 'https://datahub.io/core/s-and-p-500'
        self.DATAHUB_URL = 'https://datahub.io'

        self.YAHOO_URL = 'https://finance.yahoo.com/'

        self.OPTIONS = Options()
        self.OPTIONS.add_argument('--headless')
        self.OPTIONS.add_argument('--disable-gpu') 
        
        nltk.download(['punkt', 'wordnet', 'stopwords', 'omw-1.4', 'vader_lexicon'])
        
        
    def get_response(self, url, headers, cookies={}):
        r = httpx.get(url, cookies=cookies, headers=headers, follow_redirects=True)
        soup = bs(r.content)
        return soup
    
    def get_card_titles(self):
        titles = []
        a_tags = self.soup.find_all('a', class_='card')
        for i in range(len(a_tags)):
            titles.append(a_tags[i].span.text)
        return titles

    def get_card_urls(self):
        urls = []
        a_tags = self.soup.find_all('a', class_='card')
        for i in range(len(a_tags)):
            urls.append(a_tags[i]['href'])
        return urls
    
    def clean_text(self, text):
        return re.sub(r'[^\w\s]', '', text)
    
    def get_url_content(self, urls):
        articles = []
        raw_articles = []
        for url in urls:
            soup = self.get_response(url, self.HEADERS)
            paragraphs = soup.find('div', class_='article-content').find_all('p')
            article = ''.join([p.get_text(separator=' ') for p in paragraphs])
            raw_articles.append(paragraphs)
            articles.append(self.clean_text(article))

        return articles, raw_articles

    def get_symbols(self, raw_articles, href_ref):
        symbol_list = []
        for p_list in raw_articles:
            article_symbols = []
            for p in p_list:
                symbols = [a_tag.text for a_tag in p.find_all('a') if re.search(href_ref, a_tag['href'])]
                if symbols:
                    for symbol in symbols:
                        article_symbols.append(symbol)
            symbol_list.append(article_symbols)
        return symbol_list
    
    def get_datahub_csv(self, URL, filename):
        soup = self.get_response(URL, self.HEADERS)
        download_url = soup.find_all('table')[1].find_all('a')[1]['href']
        download_url = self.DATAHUB_URL + download_url
        with open(self.DATA_LOCATION + filename, 'wb') as f:
            f.write(requests.get(download_url).content)
            
    def get_article_links(self, driver):
        links = driver.find_elements(By.TAG_NAME, 'a')
        return links
    
    def get_yahoo_news_articles_df(self):
        yahoo_news_driver = webdriver.Chrome(options=self.OPTIONS)

        yahoo_news_driver.get(self.YAHOO_URL)
        time.sleep(1)
        yahoo_news_driver.find_element(By.CLASS_NAME, 'reject-all').click()
        actions = ActionChains(yahoo_news_driver)
        for i in range(1500):
            actions.scroll_by_amount(0, 500)
        actions.perform()
        actions.reset_actions()

        links = self.get_article_links(yahoo_news_driver)
        df = pd.DataFrame({'title': [link.text for link in links], 
                           'url': [link.get_attribute('href') for link in links], 
                           'classes': [link.get_attribute('class') for link in links]})

        yahoo_news_driver.quit()
        df = df[df['url'] \
                .str.contains('/news/') & df['classes'].str.contains('js-content-viewer')]\
                .drop_duplicates(subset='url', keep='last')\
                .reset_index(drop=True)
        return df
    
    def progress_bar(self, current, total, bar_length=20):
        fraction = current / total

        arrow = int(fraction * bar_length - 1) * '-' + '>'
        padding = int(bar_length - len(arrow)) * ' '

        ending = '\n' if current == total else '\r'

        print(f'Progress: [{arrow}{padding}] {int(fraction*100)}%', end=ending)
    
    def scrape_articles(self, urls):
        articles = []
        raw_articles = []
        yahoo_driver = webdriver.Chrome(options=self.OPTIONS)
        yahoo_driver.get(self.YAHOO_URL)
        yahoo_driver.find_element(By.CLASS_NAME, 'reject-all').click()

        for index, url in enumerate(urls):
            yahoo_driver.get(url)
            try:
                yahoo_driver.find_element(By.CLASS_NAME, 'collapse-button').click()
            except:
                pass
            article_p_elements = yahoo_driver.find_element(By.CLASS_NAME, 'caas-body').find_elements(By.TAG_NAME, 'p')
            article_by_paragraph = [p.text for p in article_p_elements if p.text!='']

            raw_article = [bs(element.get_attribute('outerHTML'), "html.parser") for element in article_p_elements]
            article = " ".join(article_by_paragraph)
            clean_article = self.clean_text(article)
            articles.append(clean_article)
            raw_articles.append(raw_article)
            self.progress_bar(index+1, len(urls))


        yahoo_driver.quit()

        return articles, raw_articles

    def clean_and_tokenize(self, articles):
        lemmatizer = WordNetLemmatizer()

        tokenized_articles = []
        for article in articles:
            tokenized_article = nltk.word_tokenize(article)
            tokenized_article = [token for token in tokenized_article if token not in stopwords.words('english')]
            tokenized_article = [lemmatizer.lemmatize(token) for token in tokenized_article]

            tokenized_articles.append(tokenized_article)

        return tokenized_articles
    
    def download_yahoo_data(self, file_path):
        """
        Downloadt gegevens van Yahoo Finance voor tickers die zijn vermeld in het opgegeven CSV-bestand.

        Parameters:
        - file_path (str): Bestandspad naar het CSV-bestand met ticker-symbolen.

        Returns:
        - pd.DataFrame: Gefilterd DataFrame met gedownloade gegevens.
        """
        df = pd.read_csv(file_path)
        symbol_list = df.iloc[:, 0].tolist()

        download = yf.download(symbol_list, group_by="ticker")
        data = download.copy()

        filtered_data = data.dropna(axis=1, how='all')
        filtered_data.columns = filtered_data.columns.remove_unused_levels()

        remaining_tickers = list(filtered_data.columns.levels[0])
        missing_tickers = list(set(symbol_list) - set(remaining_tickers))

#         print("Ontbrekende tickers en het aantal:", missing_tickers, len(missing_tickers))
#         display(filtered_data, filtered_data.shape)

        return filtered_data    
    
    def combine_data(self, api_df, investopedia_df, yahoo_df, time_data):
        df1 = investopedia_df.drop(['symbols'], axis=1)
        df2 = yahoo_df.drop('classes', axis=1)
        text_df = pd.concat([df1, df2])
        text_df = text_df.reset_index(drop=True)
        
        # tokenize the articles and add them to a DataFrame
        text_df['tokenized_article'] = self.clean_and_tokenize(text_df['article'])
        
        # apply sentiment analysis and add it to the DataFrame
        analyzer = SentimentIntensityAnalyzer()
        text_df['positivity_score'] = [analyzer.polarity_scores(article)['pos'] for article in text_df['article']]
        text_df['negativity_score'] = [analyzer.polarity_scores(article)['neg'] for article in text_df['article']]
        
        # get statistics from positive and negative sentiment analysis
        pos = text_df.describe().T[:1].reset_index(drop=True)
        neg = text_df.describe().T[1:].reset_index(drop=True)
        
        # rename columns in pos and neg so they can be concattinated
        pos.columns = ['pos_' + col for col in pos.columns]
        neg.columns = ['neg_' + col for col in neg.columns]
        
        # concattinate the positve and negative sentiment analysis statistics to a 1 row DataFrame
        pos_neg_row = pd.concat([pos, neg], axis=1)
        
        api_df = api_df.iloc[::-1].reset_index(drop=False)

        combined_df = pd.concat([api_df, pos_neg_row], axis=1)
        
        final_df = combined_df.merge(time_data, how='left', on='Date')
        
        return combined_df

    def extract(self):
#-------------------------------------------------------------------------------------------------------
        # get site data
        self.soup = self.get_response(self.HOMEPAGE_URL, self.HEADERS)

#-------------------------------------------------------------------------------------------------------   
        # download csv data
        # get_datahub_csv(self.VUSA_DOWNLOAD_URL, 'vusa_holdings.csv')
        # get_datahub_csv(self.SP_DOWNLOAD_URL, 'time_data.csv')
       
        # get csv data
        self.vusa_df = pd.read_csv(self.DATA_LOCATION + 'vusa_holdings.csv')
        self.time_data_df = pd.read_csv(self.DATA_LOCATION + 'time_data.csv')

#-------------------------------------------------------------------------------------------------------
        # get yahoo articles with titles, urls and css classes
        self.yahoo_news_df = self.get_yahoo_news_articles_df()
    
        # scrape news articles from yahoo article urls
        self.yahoo_news_articles, self.raw_yahoo_news_articles = self.scrape_articles(self.yahoo_news_df['url'])
        
#-------------------------------------------------------------------------------------------------------
        # get api data
        self.api_df = self.download_yahoo_data('Datasets/vusa_holdings.csv')
    
#-------------------------------------------------------------------------------------------------------  
    def transform(self):        
        
#-------------------------------------------------------------------------------------------------------          
        # filter site data
        titles = self.get_card_titles()
        urls = self.get_card_urls()
        
#-------------------------------------------------------------------------------------------------------
        # filter symbols from the vusa_df csv data
        symbols_in_vusa = self.vusa_df['Symbol'].tolist()

#-------------------------------------------------------------------------------------------------------
        # add scraped articles to DataFrame
        self.yahoo_news_df['article'] = self.yahoo_news_articles

#-------------------------------------------------------------------------------------------------------
        # transform filtered data to DataFrame(investopedia_df)
        investopedia_df = pd.DataFrame({'url': urls, 'title': titles})
        
        # get articles from urls
        articles, raw_articles = self.get_url_content(investopedia_df['url'])
        
        # get symbols from article
        symbols = self.get_symbols(raw_articles, 'widgetsymbol')
        
        # add articles and symbols to DataFrame(investopedia_df)
        investopedia_df['article'] = articles
        investopedia_df['symbols'] = symbols
        
        # filter DataFrame(investopedia_df) to only have symbols that occur in the symbols_in_vusa
        investopedia_df = investopedia_df[investopedia_df['symbols'].apply(lambda symbols: any(symbol in symbols_in_vusa for symbol in symbols))]
        
#-------------------------------------------------------------------------------------------------------    
        # combine DataFrames
        self.df = self.combine_data(self.api_df, investopedia_df, self.yahoo_news_df, self.time_data_df)
        
#-------------------------------------------------------------------------------------------------------   
    def load(self):
        pass

In [273]:
pipeline = Pipeline()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mark\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mark\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mark\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Mark\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Mark\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
pipeline.extract()
pipeline.transform()

In [ ]:
pipeline.df